In [ ]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import cv2                                      
%matplotlib inline       
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint  
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator               
from tqdm import tqdm


2022-11-28 18:27:07.774401: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

In [ ]:
len(train_targets)

6680

In [ ]:
def display_img(img_path):
    img = cv2.imread(img_path)
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imgplot = plt.imshow(cv_rgb)
    return imgplot

In [ ]:
## find the list of labels and save them as a pandas data 
labels_train = []
labels_test = []

for i in range(train_files.shape[0]):
    labels_train.append(dog_names[np.argmax(train_targets[i])])
    
for i in range(test_files.shape[0]):
    labels_test.append(dog_names[np.argmax(test_targets[i])])

# CNN model from scratch


### Pre-process the Data
When using TensorFlow as backend, Keras CNNs require a 4D array (which we'll also refer to as a 4D tensor) as input, with shape

(nb_samples,rows,columns,channels),
 
where nb_samples corresponds to the total number of images (or samples), and rows, columns, and channels correspond to the number of rows, columns, and channels for each image, respectively.

The path_to_tensor function below takes a string-valued file path to a color image as input and returns a 4D tensor suitable for supplying to a Keras CNN. The function first loads the image and resizes it to a square image that is  224×224  pixels. Next, the image is converted to an array, which is then resized to a 4D tensor. In this case, since we are working with color images, each image has three channels. Likewise, since we are processing a single image (or sample), the returned tensor will always have shape

(1,224,224,3).
 
The paths_to_tensor function takes a numpy array of string-valued image paths as input and returns a 4D tensor with shape

(nb_samples,224,224,3).
 
Here, nb_samples is the number of samples, or number of images, in the supplied array of image paths. It is best to think of nb_samples as the number of 3D tensors (where each 3D tensor corresponds to a different image) in your dataset!

In [ ]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

We rescale the images by dividing every pixel in every image by 255.

In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|████████████████████████████████████████| 836/836 [00:05<00:00, 155.96it/s]


In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()

# architucture

# layer 1
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=2))

# layer 2
model.add(Conv2D(filters=32, kernel_size=2 , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=2))

# layer 3
model.add(Conv2D(filters=64, kernel_size=2 , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=2))

# layer 4
model.add(Conv2D(filters=64 , kernel_size=2 , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.4))

# layer 5
model.add(Conv2D(filters=128 , kernel_size=2 , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.4))

# 2 fully connected layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(133,activation='softmax'))

model.summary()

2022-11-27 17:02:02.576497: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      4128      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 64)        8256      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 28, 28, 64)       0

### Compile the Model

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint  


epochs = 30


checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

history = model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Epoch 1/30
334/334 [==============================] - ETA: 0s - loss: 4.8785 - accuracy: 0.0123
Epoch 1: val_loss improved from inf to 4.79596, saving model to saved_models/weights.best.from_scratch.hdf5
334/334 [==============================] - 151s 448ms/step - loss: 4.8785 - accuracy: 0.0123 - val_loss: 4.7960 - val_accuracy: 0.0180
Epoch 2/30
334/334 [==============================] - ETA: 0s - loss: 4.6784 - accuracy: 0.0256
Epoch 2: val_loss improved from 4.79596 to 4.53538, saving model to saved_models/weights.best.from_scratch.hdf5
334/334 [==============================] - 146s 436ms/step - loss: 4.6784 - accuracy: 0.0256 - val_loss: 4.5354 - val_accuracy: 0.0335
Epoch 3/30
334/334 [==============================] - ETA: 0s - loss: 4.4151 - accuracy: 0.0463
Epoch 3: val_loss improved from 4.53538 to 4.28848, saving model to saved_models/weights.best.from_scratch.hdf5
334/334 [==============================] - 145s 433ms/step - loss: 4.4151 - accuracy: 0.0463 - val_loss: 4.288

Epoch 28/30
334/334 [==============================] - ETA: 0s - loss: 2.1762 - accuracy: 0.4409
Epoch 28: val_loss did not improve from 3.62333
334/334 [==============================] - 131s 393ms/step - loss: 2.1762 - accuracy: 0.4409 - val_loss: 3.7084 - val_accuracy: 0.1377
Epoch 29/30
334/334 [==============================] - ETA: 0s - loss: 2.1467 - accuracy: 0.4584
Epoch 29: val_loss did not improve from 3.62333
334/334 [==============================] - 124s 373ms/step - loss: 2.1467 - accuracy: 0.4584 - val_loss: 3.6495 - val_accuracy: 0.1653
Epoch 30/30
334/334 [==============================] - ETA: 0s - loss: 2.1164 - accuracy: 0.4590
Epoch 30: val_loss did not improve from 3.62333
334/334 [==============================] - 134s 402ms/step - loss: 2.1164 - accuracy: 0.4590 - val_loss: 3.8468 - val_accuracy: 0.1186


In [ ]:
model.save('saved_models/cnn_model.h5')

### Load the Model with the Best Validation Loss

In [ ]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

### Test the model

In [ ]:
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 26ms/step
Test accuracy: 15.3110%


### Data Augmentation

In [ ]:
## data augmentation
from keras.preprocessing.image import ImageDataGenerator

## create a generator that rotate, zoom and flip the images
traingen = ImageDataGenerator(rotation_range=40,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rescale=1/255,
        shear_range=0.04,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip= False,
        fill_mode='nearest')
validgen = ImageDataGenerator(rescale=1/255)

## apply the generator on test and valid sets
traingen.fit(train_tensors)
validgen.fit(valid_tensors)

df_training = traingen.flow(train_tensors , train_targets , batch_size = 20)
df_validation = validgen.flow(valid_tensors , valid_targets, batch_size = 20)

In [ ]:
from keras.optimizers import Adam
model.compile(optimizer= Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='saved_models/weights.initial_scratch_model_aug.hdf5', verbose = 0, save_best_only=True)
model.fit_generator(df_training, epochs = 25 , steps_per_epoch = train_tensors.shape[0]//32 , 
                   callbacks=[checkpointer] , verbose=1 , 
                   validation_data= df_validation , validation_steps = valid_tensors.shape[0]//32)

/var/folders/5c/_m6k74_557zg9wwfqkptf4540000gn/T/ipykernel_37599/814489786.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(df_training, epochs = 25 , steps_per_epoch = train_tensors.shape[0]//32 ,


Epoch 1/25
208/208 [==============================] - 109s 521ms/step - loss: 4.9428 - accuracy: 0.0075 - val_loss: 4.9024 - val_accuracy: 0.0077
Epoch 2/25
208/208 [==============================] - 101s 483ms/step - loss: 4.8863 - accuracy: 0.0113 - val_loss: 4.8863 - val_accuracy: 0.0115
Epoch 3/25
208/208 [==============================] - 100s 478ms/step - loss: 4.8832 - accuracy: 0.0089 - val_loss: 4.8668 - val_accuracy: 0.0115
Epoch 4/25
208/208 [==============================] - 100s 481ms/step - loss: 4.8820 - accuracy: 0.0096 - val_loss: 4.8740 - val_accuracy: 0.0115
Epoch 5/25
208/208 [==============================] - 96s 459ms/step - loss: 4.8784 - accuracy: 0.0075 - val_loss: 4.8747 - val_accuracy: 0.0038
Epoch 6/25
208/208 [==============================] - 100s 479ms/step - loss: 4.8733 - accuracy: 0.0113 - val_loss: 4.8756 - val_accuracy: 0.0096
Epoch 7/25
208/208 [==============================] - 102s 488ms/step - loss: 4.8735 - accuracy: 0.0099 - val_loss: 4.8709 - 

In [ ]:
## strangeley, the data augmenation accuracy becomes lower than original data. It usually causes overfitting, but in my case it is underfitting.
## need to work more on the generator 
dog_breed_predictions_aug = [np.argmax(model.predict(np.expand_dims(tensor, axis = 0))) for tensor in test_tensors]

test_accuracy_aug = 100*np.sum(np.array(dog_breed_predictions_aug)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions_aug)
print('Test accuracy with Data Augmentation: %.f%%' % test_accuracy_aug)


1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 34ms/step
Test accuracy with Data Augmentation: 1%
